## The "with" statement (PEP 343)

파이썬 공식 Reference 에서 발췌
> Python’s with statement supports the concept of a runtime context defined by a context manager. This is implemented using a pair of methods that allow user-defined classes to define a runtime context that is entered before the statement body is executed and exited when the statement ends: